# LA Building Permits: 2013-2019

In [1]:
import pandas as pd
import matplotlib as mpl
import geopandas as gpd
import descartes
import geojson
import json
import altair as alt
import altair_latimes as lat
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context  
alt.renderers.enable('notebook')
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.float_format = '{:,}'.format

### Read data (as of 9/9/2019), clean up column headers

In [2]:
#https://data.lacity.org/A-Prosperous-City/Building-and-Safety-Permit-Information/yv23-pmwf

permits = pd.read_csv('/Users/mhustiles/Desktop/data/LA/Building_and_Safety_Permit_Information.csv')
permits.columns = permits.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

/Users/mhustiles/.virtualenvs/jupyter/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,1,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Clean up data types

In [3]:
permits['project_number'] = permits['project_number'].astype(str)
permits['address_start'] = permits['address_start'].astype(str)
permits['address_end'] = permits['address_end'].astype(str)
permits['zip_code'] = permits['zip_code'].astype(str).replace('\.0', '', regex=True)
permits['assessor_book'] = permits['assessor_book'].astype(str).replace('\.0', '', regex=True)
permits['assessor_page'] = permits['assessor_page'].astype(str).replace('\.0', '', regex=True)
permits['license_#'] = permits['license_#'].astype(str)
permits['census_tract'] = permits['census_tract'].astype(str)
permits['council_district'] = permits['council_district'].astype(str)
permits['existing_code'] = permits['existing_code'].astype(str)
permits['proposed_code'] = permits['proposed_code'].astype(str)
permits['issue_date'] = pd.to_datetime(permits['issue_date'], format='%m/%d/%Y')
permits['status_date'] = pd.to_datetime(permits['status_date'], format='%m/%d/%Y')
permits['year-issued'] = permits['issue_date'].dt.year.astype(str)
permits['month-issued'] = permits['issue_date'].dt.month.astype(str)
permits['year-issued'] = permits['issue_date'].dt.year.astype(str)
permits['weekday-issued'] = permits['issue_date'].dt.weekday_name.astype(str)
permits['year-status'] = permits['status_date'].dt.year.astype(str)
permits['month-status'] = permits['status_date'].dt.month.astype(str)
permits['year-status'] = permits['status_date'].dt.year.astype(str)
permits['weekday-status'] = permits['status_date'].dt.weekday_name.astype(str)
permits['floor_area-l.a._zoning_code_definition'] = permits['floor_area-l.a._zoning_code_definition'].astype(float)
permits['area'] = permits['floor_area-l.a._zoning_code_definition'].astype(float)

### How many permits total?

In [4]:
# how many? 
len(permits)

1037228

### Filter list of all permits to just new construction on homes

In [5]:
new_home_permits = permits[(permits['permit_type'] == 'Bldg-New') & (permits['permit_sub-type'] == '1 or 2 Family Dwelling')]

In [6]:
#how many new home permits is that? 
len(new_home_permits)

21354

### Filter list of new contruction/homes to our zip codes

In [7]:
belair_permits = new_home_permits[new_home_permits['zip_code'].isin(['90077'])]
ourpermits = new_home_permits[new_home_permits['zip_code'].isin(['90169', '90210', '90077', '90049', '90272'])]

In [8]:
# how many new home permits in bel air? 
len(belair_permits)

273

In [9]:
# how many new home permits in the five zip codes? 
len(ourpermits)

1682

In [10]:
### All new_home_permits in all zip codes

In [11]:
new_home_permits.area.median()

1829.0

In [45]:
ourpermits.groupby(['year-issued', 'zip_code']).median()[['area']].round(0).sort_values(by='area', ascending=False)

,,area
year-issued,zip_code,
2018,90077,"6,368.0"
2016,90077,"5,105.0"
2014,90272,"4,634.0"
2017,90272,"4,310.0"
2014,90049,"4,296.0"
2019,90272,"4,266.0"
2017,90077,"4,226.0"
2013,90272,"4,220.0"
2018,90272,"4,161.0"


In [43]:
belair_permits[['year-issued', 'address_start', 'street_name', 'area']].sort_values(by='area', ascending=False)

,year-issued,address_start,street_name,area
690283,2014,944.0,AIROLE,"73,934.0"
367841,2016,800.0,TORTUOSO,"56,797.0"
9178,2017,10550.0,BELLAGIO,"49,669.0"
869223,2017,475.0,BEL-AIR,"38,202.0"
632136,2014,924.0,BEL-AIR,"32,728.0"
392356,2018,344.0,DELFERN,"30,887.0"
120011,2017,960.0,STRADELLA,"30,671.0"
143637,2016,788.0,TORTUOSO,"30,342.0"
600316,2016,780.0,TORTUOSO,"29,341.0"
723874,2017,418.0,NORTH FARING,"26,420.0"


### Median area for new homes in all zip codes

In [30]:
median_area_all = new_home_permits.groupby('zip_code').mean()[['area']].round(0).sort_values(by='area', ascending=False)

### Median area for new homes in all zip codes, top 10

In [31]:
median_area_all.head(10)

,area
zip_code,
90077,"6,705.0"
90210,"5,779.0"
90067,"5,756.0"
90069,"5,595.0"
90272,"4,612.0"
90049,"4,344.0"
90024,"3,679.0"
91436,"3,464.0"
91423,"3,410.0"


### Basic descriptives about our permits

In [32]:
ourpermits.describe(include=['number']).round()
belair_permits.describe(include=['number']).round()

,valuation,floor_area-l.a._zoning_code_definition,#_of_residential_dwelling_units,#_of_accessory_dwelling_units,#_of_stories,floor_area-l.a._building_code_definition,area
count,273.0,267.0,160.0,1.0,273.0,273.0,267.0
mean,"1,220,309.0","6,705.0",1.0,1.0,2.0,"7,908.0","6,705.0"
std,"1,884,459.0","9,432.0",0.0,nan,1.0,"11,170.0","9,432.0"
min,0.0,0.0,0.0,1.0,0.0,68.0,0.0
25%,"50,000.0",470.0,1.0,1.0,1.0,564.0,470.0
50%,"500,000.0","3,358.0",1.0,1.0,2.0,"3,697.0","3,358.0"
75%,"1,582,720.0","9,810.0",1.0,1.0,2.0,"11,306.0","9,810.0"
max,"12,200,000.0","73,934.0",1.0,1.0,4.0,"67,845.0","73,934.0"


In [33]:
ourpermits.columns

Index(['assessor_book', 'assessor_page', 'assessor_parcel', 'tract', 'block',
       'lot', 'reference_#_old_permit_#', 'pcis_permit_#', 'status',
       'status_date', 'permit_type', 'permit_sub-type', 'permit_category',
       'project_number', 'event_code', 'initiating_office', 'issue_date',
       'address_start', 'address_fraction_start', 'address_end',
       'address_fraction_end', 'street_direction', 'street_name',
       'street_suffix', 'suffix_direction', 'unit_range_start',
       'unit_range_end', 'zip_code', 'work_description', 'valuation',
       'floor_area-l.a._zoning_code_definition',
       '#_of_residential_dwelling_units', '#_of_accessory_dwelling_units',
       '#_of_stories', 'contractor's_business_name', 'contractor_address',
       'contractor_city', 'contractor_state', 'license_type', 'license_#',
       'principal_first_name', 'principal_middle_name', 'principal_last_name',
       'license_expiration_date', 'applicant_first_name',
       'applicant_last_name'

In [41]:
belair_permits['area'].median()

3358.0

In [37]:
permit_types = belair_permits.groupby(['permit_type']).agg('size').sort_values(ascending=False).reset_index()

In [38]:
permit_sub_types = belair_permits.groupby(['permit_sub-type']).agg('size').sort_values(ascending=False).reset_index()

In [39]:
permit_types

,permit_type,0
0,Bldg-New,273


In [40]:
permit_sub_types

,permit_sub-type,0
0,1 or 2 Family Dwelling,273


### How many have been issued?

In [ ]:
zip_count = ourpermits['year-issued'].groupby(ourpermits['zip_code'])
year_count = ourpermits['zip_code'].groupby(ourpermits['year-issued'])
license_count = ourpermits['zip_code'].groupby(ourpermits['license_type'])

In [ ]:
year_count.count()

In [ ]:
zip_count.count()

In [ ]:
license_count.count()